##  Linear Regression

In [ ]:
# 1) Design Model (input, output size, forward pass)
# 2) Construct loss and optimizer
# 3) Training loop
# -forward pass: compute prediction and loss
# -backward pass: gradients
# -update weights
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

# 0) prepare data
X_numpy,y_numpy = datasets.make_regression(n_samples = 100, n_features = 1, noise = 20, random_state = 1)
X = torch.from_numpy(X_numpy.astype(np.float32))
y = torch.from_numpy(y_numpy.astype(np.float32))
y = y.view(y.shape[0], 1)

n_samples, n_features = X.shape
# 1) model
input_size = n_features
ouput_size = 1
model = nn.Linear(input_size,ouput_size)

# 2) loss and optimizer
learning_rate = 0.01
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

# 3) training loop
num_epochs = 100
for epoch in range(num_epochs):
    # forward pass and loss
    y_predicted = model(X)
    loss = criterion(y_predicted, y)
    
    #backward pass
    loss.backward()
    
    #update
    optimizer.step()
    
    optimizer.zero_grad
    
    if (epoch+1)%10 == 0:
        print(f'epoch {epoch+1}, loss = {loss.item():.4f}')
        
# plot
predicted = model(X).detach().numpy()
plt.plot(X_numpy, y_numpy, 'ro')
plt.plot(X_numpy, predicted, 'b')
plt.show

In [ ]:
# 1) Design Model - input,output size, forward pass
# 2) loss and optimizer
# 3) backward pass
# forward pass: compute prediction and loss
# backward pass: gradients
# update weights
import torch
import numpy as np
import torch.nn as nn
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 0) data preparation
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1234)

#scale
sc = StandardScaler()         #this will make our features to have zero mean and unit variance and always recommended to do
                            #when we deal with logistic regression
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)


# 1) model
# f = wx + b, sigmoid
class LogisticRegression(nn.Module):
    
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.Linear = nn.Linear(n_input_features, 1)
        
    def forward(self,x):
        y_predicted = torch.sigmoid(self.Linear(x))          #sigmoid function will return a value betweeen 0 and 1
        return y_predicted
    
model = LogisticRegression(n_features)


# 2) loss and optimizer
learning_rate = 0.01
criterion = nn.BCELoss()                              #creates a criterion that measures the Binary Cross Entropy Loss
                                        # between the target and the output
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

# 3) training loop
num_epochs = 100
for epoch in range(num_epochs):
    #forward pass and loss
    y_predicted = model(X_train)
    loss = criterion(y_predicted, y_train)
    
    #backward pass
    loss.backward()
    
    #updates
    optimizer.step()
    
    #zero gradients
    optimizer.zero_grad()
    
    if(epoch+1)%10 == 0:
        print(f'epoch: {epoch+1}, loss= {loss.item():.4f}')
        
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')

## Dataset and DataLoader

In [7]:
import torch
import torchvision
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
import math

class WineDataset(Dataset):
    
    def __init__(self):
        xy = np.loadtxt("C:\\Users\\User\\Desktop\\python\\WineQT.csv",delimiter = ',', dtype = np.float32, skiprows = 1 )
        self.x = torch.from_numpy(xy[:, 1:])
        self.y = torch.from_numpy(xy[:, [0]])  #n_samples, 1
        self.n_samples = xy.shape[0]
        
    def __getitem__(self,index):
        #dataset[0]
        return self.x[index], self.y[index]
        
    def __len__(self):
        #len(dataset)
        return self.n_samples
    
dataset = WineDataset()
first_data = dataset[0]
features, labels = first_data
print(features, labels)

tensor([ 0.7000,  0.0000,  1.9000,  0.0760, 11.0000, 34.0000,  0.9978,  3.5100,
         0.5600,  9.4000,  5.0000,  0.0000]) tensor([7.4000])


In [4]:
# importing the required libraries
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# defining the Dataset class
class data_set(Dataset):
	def __init__(self):
		numbers = list(range(0, 100, 1))
		self.data = numbers

	def __len__(self):
		return len(self.data)

	def __getitem__(self, index):
		return self.data[index]


dataset = data_set()

# implementing dataloader on the dataset and printing per batch
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)
for i, batch in enumerate(dataloader):
	print(i, batch)


0 tensor([30, 75, 40, 58, 89, 50, 36, 14, 25, 56])
1 tensor([ 2, 76, 53, 68, 61,  0, 80, 64, 67, 52])
2 tensor([94, 60, 19, 99, 66,  8, 87, 23, 42, 91])
3 tensor([70,  5, 77, 24, 73,  1, 49, 46, 63, 47])
4 tensor([20, 95, 71, 39, 12, 55, 21,  6, 41, 28])
5 tensor([74, 62,  7, 54, 13, 31, 93, 98, 81, 27])
6 tensor([78,  9, 43,  4, 45, 32, 22, 92, 65, 88])
7 tensor([72, 90, 29, 51, 69, 26, 44, 16,  3, 82])
8 tensor([59, 86, 15, 48, 85, 57, 11, 38, 97, 79])
9 tensor([83, 18, 10, 33, 37, 34, 17, 35, 96, 84])


In [11]:
import torch
import torchvision
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
import math

class WineDataset(Dataset):
    
    def __init__(self):
        xy = np.loadtxt("C:\\Users\\User\\Desktop\\python\\WineQT.csv",delimiter = ',', dtype = np.float32, skiprows = 1 )
        self.x = torch.from_numpy(xy[:, 1:])
        self.y = torch.from_numpy(xy[:, [0]])  #n_samples, 1
        self.n_samples = xy.shape[0]
        
    def __getitem__(self,index):
        #dataset[0]
        return self.x[index], self.y[index]
        
    def __len__(self):
        #len(dataset)
        return self.n_samples
    
dataset = WineDataset()
dataloader = DataLoader(dataset = dataset, batch_size = 4, shuffle = True, num_workers = 0 )

dataiter = iter(dataloader)
data = dataiter.next()
features, labels = data
print(features, labels)

#training loop 
num_epochs = 2
total_samples = len(dataset)
n_iterations = math.ceil(total_samples/4)
print(total_samples, n_iterations)

for epoch in range(num_epochs):
    for i, (inputs,labels) in enumerate(dataloader):
        #forward backward, update
        if (i+1) % 5 == 0:
            print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_iterations}, inputs {inputs.shape}')

            

tensor([[4.7000e-01, 2.0000e-02, 1.3000e+00, 3.4000e-02, 1.8000e+01, 4.4000e+01,
         9.9210e-01, 3.9000e+00, 6.2000e-01, 1.2800e+01, 6.0000e+00, 6.9500e+02],
        [6.8000e-01, 2.0000e-02, 1.8000e+00, 8.7000e-02, 2.1000e+01, 9.4000e+01,
         9.9440e-01, 3.5400e+00, 5.2000e-01, 1.0000e+01, 5.0000e+00, 1.4600e+02],
        [6.1000e-01, 8.0000e-02, 2.1000e+00, 7.1000e-02, 1.6000e+01, 2.4000e+01,
         9.9376e-01, 3.5600e+00, 7.7000e-01, 1.1100e+01, 6.0000e+00, 8.3100e+02],
        [9.0000e-01, 3.4000e-01, 6.6000e+00, 1.1200e-01, 2.3000e+01, 9.9000e+01,
         1.0029e+00, 3.2200e+00, 6.8000e-01, 9.3000e+00, 5.0000e+00, 8.8900e+02]]) tensor([[ 5.1000],
        [ 5.8000],
        [ 5.9000],
        [10.7000]])
1143 286
epoch 1/2, step 5/286, inputs torch.Size([4, 12])
epoch 1/2, step 10/286, inputs torch.Size([4, 12])
epoch 1/2, step 15/286, inputs torch.Size([4, 12])
epoch 1/2, step 20/286, inputs torch.Size([4, 12])
epoch 1/2, step 25/286, inputs torch.Size([4, 12])
epoch 1

## Dataset Transforms

In [7]:
import torch
import torchvision
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
import math

class WineDataset(Dataset):
    
    def __init__(self, transform = None):
        xy = np.loadtxt("C:\\Users\\User\\Desktop\\python\\WineQT.csv",delimiter = ',', dtype = np.float32, skiprows = 1 )
        self.x = torch.from_numpy(xy[:, 1:])
        self.y = torch.from_numpy(xy[:, [0]])  #n_samples, 1
        self.n_samples = xy.shape[0]
        
        self.transform = transform
        
    def __getitem__(self,index):
        #dataset[0]
        sample =  self.x[index], self.y[index]
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample
    
    def __len__(self):
        #len(dataset)
        return self.n_samples
    
    
class ToTensor:
    def __call__(self,sample):
        inputs, targets = sample
        return torch.from_numpy(inputs), torch.from_numpy(targets)
    
class MulTransform:
    def __call__(self,factor):
        self.factor = factor
        
    def __call__(self,sample):
        inputs, target = sample
        inputs *= sample
        return inputs, target
    
dataset = WineDataset(transform = ToTensor())
first_data = dataset[0]
features, labels = first_data
print(features)
print(type(features), type(labels))

composed = torchvision.transforms.compose([ToTensor(), MulTransform(2)])
dataset = WineDataset(transform = composed)
first_data = dataset[0]
features, labels = first_data
print(features)
print(type(features), type(labels))

TypeError: expected np.ndarray (got Tensor)

In [9]:
'''
Transforms can be applied to PIL images, tensors, ndarrays, or custom data
during creation of the DataSet
complete list of built-in transforms: 
https://pytorch.org/docs/stable/torchvision/transforms.html
On Images
---------
CenterCrop, Grayscale, Pad, RandomAffine
RandomCrop, RandomHorizontalFlip, RandomRotation
Resize, Scale
On Tensors
----------
LinearTransformation, Normalize, RandomErasing
Conversion
----------
ToPILImage: from tensor or ndrarray
ToTensor : from numpy.ndarray or PILImage
Generic
-------
Use Lambda 
Custom
------
Write own class
Compose multiple Transforms
---------------------------
composed = transforms.Compose([Rescale(256),
                               RandomCrop(224)])
'''

import torch
import torchvision
from torch.utils.data import Dataset
import numpy as np


class WineDataset(Dataset):

    def __init__(self, transform=None):
        xy = np.loadtxt('C:\\Users\\User\\Desktop\\python\\WineQT.csv', delimiter=',', dtype=np.float32, skiprows=1)
        self.n_samples = xy.shape[0]

        # note that we do not convert to tensor here
        self.x_data = xy[:, 1:]
        self.y_data = xy[:, [0]]

        self.transform = transform

    def __getitem__(self, index):
        sample = self.x_data[index], self.y_data[index]

        if self.transform:
            sample = self.transform(sample)

        return sample

    def __len__(self):
        return self.n_samples

# Custom Transforms
# implement __call__(self, sample)
class ToTensor:
    # Convert ndarrays to Tensors
    def __call__(self, sample):
        inputs, targets = sample
        return torch.from_numpy(inputs), torch.from_numpy(targets)

class MulTransform:
    # multiply inputs with a given factor
    def __init__(self, factor):
        self.factor = factor

    def __call__(self, sample):
        inputs, targets = sample
        inputs *= self.factor
        return inputs, targets

print('Without Transform')
dataset = WineDataset()
first_data = dataset[0]
features, labels = first_data
print(type(features), type(labels))
print(features, labels)

print('\nWith Tensor Transform')
dataset = WineDataset(transform=ToTensor())
first_data = dataset[0]
features, labels = first_data
print(type(features), type(labels))
print(features, labels)

print('\nWith Tensor and Multiplication Transform')
composed = torchvision.transforms.Compose([ToTensor(), MulTransform(4)])
dataset = WineDataset(transform=composed)
first_data = dataset[0]
features, labels = first_data
print(type(features), type(labels))
print(features, labels)

Without Transform
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
[ 0.7     0.      1.9     0.076  11.     34.      0.9978  3.51    0.56
  9.4     5.      0.    ] [7.4]

With Tensor Transform
<class 'torch.Tensor'> <class 'torch.Tensor'>
tensor([ 0.7000,  0.0000,  1.9000,  0.0760, 11.0000, 34.0000,  0.9978,  3.5100,
         0.5600,  9.4000,  5.0000,  0.0000]) tensor([7.4000])

With Tensor and Multiplication Transform
<class 'torch.Tensor'> <class 'torch.Tensor'>
tensor([  2.8000,   0.0000,   7.6000,   0.3040,  44.0000, 136.0000,   3.9912,
         14.0400,   2.2400,  37.6000,  20.0000,   0.0000]) tensor([7.4000])


## Softmax and Cross Entropy

In [1]:
import torch
import torch.nn as nn
import numpy as np

def softmax(x):
    return np.exp(x)/np.sum(np.exp(x), axis = 0)

x = np.array([2.0,1.0,0.1])
outputs = softmax(x)
print('Softmax numpy:',outputs)

Softmax numpy: [0.65900114 0.24243297 0.09856589]


In [2]:
x = torch.tensor([2.0,1.0,0.1])
outputs = torch.softmax(x, dim=0)
outputs

tensor([0.6590, 0.2424, 0.0986])

In [4]:
#a lot of time the softmax function is combined with the cross-entropy loss, this measures the performance of the classification model 
#whose output is the probability is between 0 and 1 and can be used in multi class problems and the loss increases if the predicted 
#probability diverges from the actual label
#So the better our prediction the lower is the loss
import torch
import torch.nn as nn
import numpy as np

#
#        -> 2.0              -> 0.65  
# Linear -> 1.0  -> Softmax  -> 0.25   -> CrossEntropy(y, y_hat)
#        -> 0.1              -> 0.1                   
#
#     scores(logits)      probabilities
#                           sum = 1.0
#

# Softmax applies the exponential function to each element, and normalizes
# by dividing by the sum of all these exponentials
# -> squashes the output to be between 0 and 1 = probability
# sum of all probabilities is 1
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

x = np.array([2.0, 1.0, 0.1])
outputs = softmax(x)
print('softmax numpy:', outputs)

x = torch.tensor([2.0, 1.0, 0.1])
outputs = torch.softmax(x, dim=0) # along values along first axis
print('softmax torch:', outputs)

# Cross entropy
# Cross-entropy loss, or log loss, measures the performance of a classification model 
# whose output is a probability value between 0 and 1. 
# -> loss increases as the predicted probability diverges from the actual label
def cross_entropy(actual, predicted):
    EPS = 1e-15
    predicted = np.clip(predicted, EPS, 1 - EPS)
    loss = -np.sum(actual * np.log(predicted))
    return loss # / float(predicted.shape[0])

# y must be one hot encoded
# if class 0: [1 0 0]
# if class 1: [0 1 0]
# if class 2: [0 0 1]
Y = np.array([1, 0, 0])
Y_pred_good = np.array([0.7, 0.2, 0.1])
Y_pred_bad = np.array([0.1, 0.3, 0.6])
l1 = cross_entropy(Y, Y_pred_good)
l2 = cross_entropy(Y, Y_pred_bad)
print(f'Loss1 numpy: {l1:.4f}')
print(f'Loss2 numpy: {l2:.4f}')

# CrossEntropyLoss in PyTorch (applies Softmax)
# nn.LogSoftmax + nn.NLLLoss
# NLLLoss = negative log likelihood loss
loss = nn.CrossEntropyLoss()
# loss(input, target)

# target is of size nSamples = 1
# each element has class label: 0, 1, or 2
# Y (=target) contains class labels, not one-hot
Y = torch.tensor([0])

# input is of size nSamples x nClasses = 1 x 3
# y_pred (=input) must be raw, unnormalizes scores (logits) for each class, not softmax
Y_pred_good = torch.tensor([[2.0, 1.0, 0.1]])
Y_pred_bad = torch.tensor([[0.5, 2.0, 0.3]])
l1 = loss(Y_pred_good, Y)
l2 = loss(Y_pred_bad, Y)

print(f'PyTorch Loss1: {l1.item():.4f}')
print(f'PyTorch Loss2: {l2.item():.4f}')

# get predictions
_, predictions1 = torch.max(Y_pred_good, 1)
_, predictions2 = torch.max(Y_pred_bad, 1)
print(f'Actual class: {Y.item()}, Y_pred1: {predictions1.item()}, Y_pred2: {predictions2.item()}')

# allows batch loss for multiple samples

# target is of size nBatch = 3
# each element has class label: 0, 1, or 2
Y = torch.tensor([2, 0, 1])

# input is of size nBatch x nClasses = 3 x 3
# Y_pred are logits (not softmax)
Y_pred_good = torch.tensor(
    [[0.1, 0.2, 3.9], # predict class 2
    [1.2, 0.1, 0.3], # predict class 0
    [0.3, 2.2, 0.2]]) # predict class 1

Y_pred_bad = torch.tensor(
    [[0.9, 0.2, 0.1],
    [0.1, 0.3, 1.5],
    [1.2, 0.2, 0.5]])

l1 = loss(Y_pred_good, Y)
l2 = loss(Y_pred_bad, Y)
print(f'Batch Loss1:  {l1.item():.4f}')
print(f'Batch Loss2: {l2.item():.4f}')

# get predictions
_, predictions1 = torch.max(Y_pred_good, 1)
_, predictions2 = torch.max(Y_pred_bad, 1)
print(f'Actual class: {Y}, Y_pred1: {predictions1}, Y_pred2: {predictions2}')

# Binary classification
class NeuralNet1(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(NeuralNet1, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, 1)  
    
    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out)
        # sigmoid at the end
        y_pred = torch.sigmoid(out)
        return y_pred

model = NeuralNet1(input_size=28*28, hidden_size=5)
criterion = nn.BCELoss()

# Multiclass problem
class NeuralNet2(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet2, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out)
        # no softmax at the end
        return out

model = NeuralNet2(input_size=28*28, hidden_size=5, num_classes=3)
criterion = nn.CrossEntropyLoss()  # (applies Softmax)

softmax numpy: [0.65900114 0.24243297 0.09856589]
softmax torch: tensor([0.6590, 0.2424, 0.0986])
Loss1 numpy: 0.3567
Loss2 numpy: 2.3026
PyTorch Loss1: 0.4170
PyTorch Loss2: 1.8406
Actual class: 0, Y_pred1: 0, Y_pred2: 1
Batch Loss1:  0.2834
Batch Loss2: 1.6418
Actual class: tensor([2, 0, 1]), Y_pred1: tensor([2, 0, 1]), Y_pred2: tensor([0, 2, 0])
